In [1]:
import csv
all_hateoffensive_tweets=[]
all_nonhate_tweets=[]
with open('hindi_dataset.tsv', newline='', encoding="utf8") as tsvfile:
    reader = csv.reader(tsvfile, delimiter='\t')
    for row in reader:
        if row[3] == "HATE" or row[3] == "OFFN":
           all_hateoffensive_tweets.append(row[1])
        else:
           all_nonhate_tweets.append(row[1]) 

In [2]:
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import TweetTokenizer
import re
import string
from matplotlib.patches import Ellipse
import matplotlib.transforms as transforms
import numpy as np 
from collections import defaultdict
words_dict = defaultdict(int)


In [3]:
len(all_nonhate_tweets)

3434

In [4]:
len(all_hateoffensive_tweets)

1232

In [5]:
all_nonhate_tweets[6]

'मुंबई में बारिश से लोगों को काफी समस्या हो रही है'

In [6]:
all_hateoffensive_tweets[6]

'तबरेज के लिए तो तेरे अमनपसंद कौम के अम्मि बेगम ने चूडियां तोडकर विध्वा विलाप किये थे!  तुम लोगों के मजार पर हिन्दू मोहल्ले के कुत्ता भी पैशाब कर दे तब भी दंगा भड़काने वाले हरामी कौम है,मन्दिर कि मूर्ति तुडवाने मे तेरा ही साजिश है  '

In [7]:
stopwords=[]
with open('hindi-stopwords.txt', 'r',encoding='UTF-8') as file:
    file=file.readlines()
    for line in file:
        stopwords.append(line.strip())

In [8]:
stopwords[0:5]

['मैं', 'मुझको', 'मेरा', 'अपने आप को', 'हमने']

In [9]:
def process_tweet(tweet):
    '''
    Input: 
        tweet: a string containing a tweet
    Output:
        tweets_clean: a list of words containing the processed tweet

    '''
    stemmer = PorterStemmer()
    # remove stock market tickers like $GE
    tweet = re.sub(r'\$\w*', '', tweet)
    # remove old style retweet text "RT"
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    # remove hyperlinks
    #tweet = re.sub(r'https?:\/\/.*[\r\n]*', '', tweet)
    tweet = re.sub(r'https?://[^\s\n\r]+', '', tweet)

    # remove hashtags
    # only removing the hash # sign from the word
    tweet = re.sub(r'#', '', tweet)
    # tokenize tweets
    tokenizer = TweetTokenizer(preserve_case=False, strip_handles=True,
                               reduce_len=True)
    tweet_tokens = tokenizer.tokenize(tweet)

    tweets_clean = []
    for word in tweet_tokens:
        if (word not in stopwords and  # remove stopwords
            word not in string.punctuation) and word != '।' and word != "...":  # remove punctuation
            # tweets_clean.append(word)
            stem_word = word  # stemming word
            tweets_clean.append(stem_word)

    return tweets_clean

In [10]:
process_tweet('तबरेज के लिए तो तेरे अमनपसंद कौम के अम्मि बेगम ने चूडियां तोडकर विध्वा विलाप किये थे!  तुम लोगों के मजार पर हिन्दू मोहल्ले के कुत्ता भी पैशाब कर दे तब भी दंगा भड़काने वाले हरामी कौम है,मन्दिर कि मूर्ति तुडवाने मे तेरा ही साजिश है  ')

['तबरेज',
 'के',
 'तेरे',
 'अमनपसंद',
 'कौम',
 'के',
 'अम्मि',
 'बेगम',
 'चूडियां',
 'तोडकर',
 'विध्वा',
 'विलाप',
 'किये',
 'लोगों',
 'के',
 'मजार',
 'हिन्दू',
 'मोहल्ले',
 'के',
 'कुत्ता',
 'पैशाब',
 'दे',
 'दंगा',
 'भड़काने',
 'हरामी',
 'कौम',
 'मन्दिर',
 'मूर्ति',
 'तुडवाने',
 'तेरा',
 'साजिश']

In [11]:
len(all_nonhate_tweets)

3434

In [12]:
len(all_hateoffensive_tweets)

1232

In [13]:
test_pos = all_nonhate_tweets[2800:]
train_pos = all_nonhate_tweets[:2800]
test_neg = all_hateoffensive_tweets[1000:]
train_neg = all_hateoffensive_tweets[:1000]

train_x = train_pos + train_neg
test_x = test_pos + test_neg

train_y = np.append(np.ones(len(train_pos)), np.zeros(len(train_neg)))
test_y = np.append(np.ones(len(test_pos)), np.zeros(len(test_neg)))

In [14]:
def count_tweets(result, tweets, ys):
    '''
    Input:
        result: a dictionary that will be used to map each pair to its frequency
        tweets: a list of tweets
        ys: a list corresponding to the sentiment of each tweet (either 0 or 1)
    Output:
        result: a dictionary mapping each pair to its frequency
    '''
    for y, tweet in zip(ys, tweets):
        for word in process_tweet(tweet):
            # define the key, which is the word and label tuple
            pair = (word,y)
            
            # if the key exists in the dictionary, increment the count
            if pair in result:
                result[pair] += 1

            # else, if the key is new, add it to the dictionary and set the count to 1
            else:
                result[pair] = 1
    return result

In [15]:
freqs = count_tweets({}, train_x, train_y)

In [16]:
def lookup(freqs, word, label):
    '''
    Input:
        freqs: a dictionary with the frequency of each pair (or tuple)
        word: the word to look up
        label: the label corresponding to the word
    Output:
        n: the number of times the word with its corresponding label appears.
    '''
    n = 0  # freqs.get((word, label), 0)

    pair = (word, label)
    if (pair in freqs):
        n = freqs[pair]

    return n

In [17]:
def train_naive_bayes(freqs, train_x, train_y):
    '''
    Input:
        freqs: dictionary from (word, label) to how often the word appears
        train_x: a list of tweets
        train_y: a list of labels correponding to the tweets (0,1)
    Output:
        logprior: the log prior. (equation 3 above)
        loglikelihood: the log likelihood of you Naive bayes equation. (equation 6 above)
    '''
    loglikelihood = {}
    logprior = 0

    # calculate V, the number of unique words in the vocabulary
    vocab = []
    for pair in freqs.keys():
        if pair[0] not in vocab:
            vocab.append(pair[0])
        
    V = len(set(vocab))    

    # calculate N_pos, N_neg, V_pos, V_neg
    N_pos = N_neg = 0
    for pair in freqs.keys():
        # if the label is positive (greater than zero)
        if pair[1] > 0:

            # Increment the number of positive words by the count for this (word, label) pair
            N_pos += lookup(freqs, pair[0], 1)

        # else, the label is negative
        else:

            # increment the number of negative words by the count for this (word,label) pair
            N_neg += lookup(freqs, pair[0], 0)
    
    # Calculate D, the number of documents
    D = len(train_y)

    # Calculate D_pos, the number of positive documents
    # Calculate D_neg, the number of negative documents

    D_pos = 0
    D_neg = 0
    for i in range(len(train_y)):
        if train_y[i] == 1:
            D_pos += 1
        else:
            D_neg += 1

    # Calculate logprior
    logprior = np.log(D_pos/D_neg)
    
    # For each word in the vocabulary...
    for word in vocab:
        # get the positive and negative frequency of the word
        freq_pos = lookup(freqs, word, 1)
        freq_neg = lookup(freqs, word, 0)
        

        # calculate the probability that each word is positive, and negative
        p_w_pos = (freq_pos+1)/(N_pos+V)
        p_w_neg = (freq_neg+1)/(N_neg+V)

        # calculate the log likelihood of the word
        loglikelihood[word] = np.log(p_w_pos/p_w_neg)

    return logprior, loglikelihood

In [18]:
logprior, loglikelihood = train_naive_bayes(freqs, train_x, train_y)
print(logprior)
print(len(loglikelihood))

1.0296194171811581
16357


In [19]:
def naive_bayes_predict(tweet, logprior, loglikelihood):
    '''
    Input:
        tweet: a string
        logprior: a number
        loglikelihood: a dictionary of words mapping to numbers
    Output:
        p: the sum of all the logliklihoods of each word in the tweet (if found in the dictionary) + logprior (a number)

    '''
    # process the tweet to get a list of words
    word_l = process_tweet(tweet)

    # initialize probability to zero
    p = 0

    # add the logprior
    p += logprior

    for word in word_l:

        # check if the word exists in the loglikelihood dictionary
        if word in loglikelihood:
            # add the log likelihood of that word to the probability
            p += loglikelihood[word]

    return p

In [20]:
def test_naive_bayes(test_x, test_y, logprior, loglikelihood, naive_bayes_predict=naive_bayes_predict):
    """
    Input:
        test_x: A list of tweets
        test_y: the corresponding labels for the list of tweets
        logprior: the logprior
        loglikelihood: a dictionary with the loglikelihoods for each word
    Output:
        accuracy: (# of tweets classified correctly)/(total # of tweets)
    """
    accuracy = 0  # return this properly
    dictss={}
    y_hats = []
    for tweet in test_x:
        # if the prediction is > 0
        if naive_bayes_predict(tweet, logprior, loglikelihood) > 0:
            # the predicted class is 1
            y_hat_i = 1
        else:
            # otherwise the predicted class is 0
            y_hat_i = 0

        # append the predicted class to the list y_hats
        y_hats.append(y_hat_i)

    # error is the average of the absolute values of the differences between y_hats and test_y
    error = np.sum(np.squeeze(np.asarray(y_hats)) != np.squeeze(test_y))/len(test_y)

    # Accuracy is 1 minus the error
    accuracy = 1 - error
    return accuracy

In [21]:
print((test_naive_bayes(test_x, test_y, logprior, loglikelihood)))

0.7066974595842956


In [22]:
for tweet in test_neg[34:39]: 
    p = naive_bayes_predict(tweet, logprior, loglikelihood)
    print(f'{tweet} -> {p:.2f}')

Sabhi party jila ikai ko tatkal prabav se bhang kar ke phir se ????????? -> 3.93
प्रियाजी, #Media420 की नमक हरामी की हद तब पार हुई ,जब पाकिस्तान में नाबालिग हिंदू बच्ची का मुस्लिम ने अपहरण करके उसे इस्लाम धर्म अपनाने के लिए मजबूर किया  , इस बारे मे ना तो मीडिया ने मुंह खोला और ना ही सरकार द्वारा कोई ठोस कदम उठाया गया| -> -16.12
कोई किसी को “गाली” दे, कोई किसी को “ठोक” दे, कोई किसी को “धमकी”  दे या कोई किसी की “हत्या”  कर दे..........राम के “नाम”  पर, अब सब चलेगा. -> -0.77
जो औरत     आतंकवादियों की मौत पर बंद कमरे मे रोती हो    उस बारडांसर विदेशी औरत ने शहीदों को श्रद्धांजलि तक नही दी    और कहते है उसके बेटे/बेटी/दामाद  को देश सौंप दो    ताकि ये  @INCIndia  @RahulGandhi  @priyankagandhi  @priyankac19  सब मिलकर    ऐसे ही पुलवामा करवाते रहें    #कमीने  -> 3.08
अब इन सूअरों को उनकी औकात दिखानी जरूरी है,  100 मैसे 98 मुल्ले नीच ओर घटिया मानसिकता वाली सोच के सुअर होते हैं ।  दिल्ली वालों एक बार हिम्मत दिखाओ, आने वाली पीढ़िया आपको दुआएं देगी।  मारो कटवो को  #TempleTerrorAttack -> -3.29


In [23]:
for tweet in test_pos[0:5]: 
    p = naive_bayes_predict(tweet, logprior, loglikelihood)
    print(f'{tweet} -> {p:.2f}')

भेनचोद तुझी लायकी आहे का महाराजां बद्दल बोलायची तुझी आयची गांड -> 5.70
ज़ी न्यूज़ खबर दे रहा है...    ' कश्मीर घाटी के मुसलमान....अमरनाथ यात्रियों का स्वागत करने को दिल से तैयार बैठे हैं '....    समझ नहीं आ रहा है कि खुश हों या घबड़ाएं... -> 7.85
सउदी अरब में नमाज के बाद योग करते असली वाले मुसलमान  -> 0.85
ज़ुल्म इतना बुरा नहीं जितनी  बुरी तुम्हारी खामोशी है...     हक़ बोलना सीखो वरना पीढियां गूंगी हो जाएंगी... -> 1.69
#AIMIM Party Floor Leader Habeeb E Millat Akbaruddin Owaisi Sahab Addressed Speach at Owaisi School Of Excellence Jamal Colony #Hyderabad! @imAkbarOwaisi   @asadowaisi -> 9.25


In [24]:
def get_ratio(freqs, word):
    '''
    Input:
        freqs: dictionary containing the words

    Output: a dictionary with keys 'positive', 'negative', and 'ratio'.
        Example: {'positive': 10, 'negative': 20, 'ratio': 0.5}
    '''
    pos_neg_ratio = {'positive': 0, 'negative': 0, 'ratio': 0.0}
    # use lookup() to find positive counts for the word (denoted by the integer 1)
    pos_neg_ratio['positive'] = lookup(freqs,word,1)
    
    # use lookup() to find negative counts for the word (denoted by integer 0)
    pos_neg_ratio['negative'] = lookup(freqs,word,0)
    
    # calculate the ratio of positive to negative counts for the word
    pos_neg_ratio['ratio'] = (lookup(freqs,word,1)+1) / (lookup(freqs,word,0)+1)
    return pos_neg_ratio


In [25]:
# UNQ_C9 GRADED FUNCTION: get_words_by_threshold

def get_words_by_threshold(freqs, label, threshold, get_ratio=get_ratio):
    '''
    Input:
        freqs: dictionary of words
        label: 1 for positive, 0 for negative
        threshold: ratio that will be used as the cutoff for including a word in the returned dictionary
    Output:
        word_list: dictionary containing the word and information on its positive count, negative count, and ratio of positive to negative counts.
        example of a key value pair:
        {'happi':
            {'positive': 10, 'negative': 20, 'ratio': 0.5}
        }
    '''
    word_list = {}

    ### START CODE HERE ###
    for key in freqs.keys():
        word= key[0]

        # get the positive/negative ratio for a word
        pos_neg_ratio = get_ratio(freqs, word)

        # if the label is 1 and the ratio is greater than or equal to the threshold...
        if label == 1 and pos_neg_ratio['ratio'] >= threshold:
        
            # Add the pos_neg_ratio to the dictionary
            word_list[word] = pos_neg_ratio

        # If the label is 0 and the pos_neg_ratio is less than or equal to the threshold...
        elif label == 0 and pos_neg_ratio['ratio'] <= threshold:
        
            # Add the pos_neg_ratio to the dictionary
            word_list[word] = pos_neg_ratio

        # otherwise, do not include this word in the list (do nothing)

    ### END CODE HERE ###
    return word_list


In [26]:
# Test your function: find negative words at or below a threshold
print(str(get_words_by_threshold(freqs, label=0, threshold=5))[0:1000])

{'text': {'positive': 1, 'negative': 0, 'ratio': 2.0}, 'वापसी': {'positive': 3, 'negative': 2, 'ratio': 1.3333333333333333}, 'भारत': {'positive': 148, 'negative': 46, 'ratio': 3.1702127659574466}, '314': {'positive': 1, 'negative': 0, 'ratio': 2.0}, 'रोका': {'positive': 2, 'negative': 1, 'ratio': 1.5}, 'सब': {'positive': 110, 'negative': 66, 'ratio': 1.6567164179104477}, 'रंडी': {'positive': 148, 'negative': 35, 'ratio': 4.138888888888889}, 'नाच': {'positive': 7, 'negative': 4, 'ratio': 1.6}, 'देखने': {'positive': 12, 'negative': 3, 'ratio': 3.25}, 'व्यस्त': {'positive': 2, 'negative': 3, 'ratio': 0.75}, 'शांतीदूत': {'positive': 1, 'negative': 0, 'ratio': 2.0}, 'के': {'positive': 1866, 'negative': 734, 'ratio': 2.540136054421769}, 'होगा': {'positive': 76, 'negative': 37, 'ratio': 2.026315789473684}, 'रंडीरोना': {'positive': 1, 'negative': 0, 'ratio': 2.0}, 'शुरू': {'positive': 32, 'negative': 13, 'ratio': 2.357142857142857}, 'देंगे': {'positive': 24, 'negative': 15, 'ratio': 1.5625}, '

In [27]:
print('Truth Predicted Tweet')
for x, y in zip(test_x, test_y):
    y_hat = naive_bayes_predict(x, logprior, loglikelihood)
    if y != (np.sign(y_hat) > 0):
        print('%d\t%0.2f\t%s' % (y, np.sign(y_hat) > 0, x))

Truth Predicted Tweet
1	0.00	इन भोश्री वाले मुल्लो के भड़वे मुहाम्मद ने   इन सूअर के बच्चो को जानवरों के साथ भी करने कि इजाज़त दी हुई है     भड़वे मुहॉम्मद का भड़वा धर्म     इन सालो को सिर्फ हवस मिटानी है फिर चाहे जैसे भी हो 
1	0.00	Breaking News :- हरियाणा के फरीदाबाद में कांग्रेस प्रवक्ता विकास चौधीर की गोली मारकर हत्या,  खट्टर जी पहले अपराध रोकिए, राम रहीम को बाद में पैरोल दिलाइएगा।
1	0.00	तेरी परेशानी क्या है सुअर  के पिल्ले मतलब दंगा नही हुआ तो तुझे वो बात भी हजम नही हुई कि वहां के हिन्दू मुस्लिम्स ने मिलकर बैठ कर समस्या का हल निकाल लिया ।  तू चाहता क्या हेबे हिंदी के बलात्कारी  
1	0.00	UP पुलिस द्वारा “पत्रकारों” की धुनाई सरकार की “तानाशाही” के साथ साथ “लोकतंत्र” के लिये एक बड़ा “ख़तरा” है,लेकिन पत्रकारों को भी अपना “ज़मीर” बेचने से पहले, कुछ थोड़ा बहुत ज़रूर सोचना चाहिये.
1	0.00	केजरीवाल का कामनवेल्थ घोटाला  सिसोदिया आज का कलमाड़ी हैं    दिल्ली में स्कूलों में कमरों के नाम पर हुई लूट का पूरा सच - जानिए कपिल मिश्रा से
1	0.00	  #चमचों के बीच गुजरते हुए #मोदी_मोदी बोल दो या फिर  #क

In [30]:
my_tweet = 'अब जनता के प्राकृतिक ऑटोमैटिक #राइफल का बारूद #नसबन्दी करके बन्द करोगे,,तो फिर खिलौना #बन्दूक तो चलाएंगे ही।।।।  इब कुत्ते कर्म करोगे तो फेर #कुत्ते की मौत ही मरोगे    #तेरा बी टेम आग्या #KejriwalAssassinationClaim'
p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print(p)

-5.118222114263541


In [29]:
my_tweet = 'कितना अच्छा समुदाय है ये हमेशा गरीब लोगो के लिए भंडारा अरेंज करता है'
p = naive_bayes_predict(my_tweet, logprior, loglikelihood)
print(p)

0.7821279987669163
